In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
data=pd.read_csv("game.csv")

In [4]:
data

,userId,game,rating
0,3,The Legend of Zelda: Ocarina of Time,4.0
1,6,Tony Hawk's Pro Skater 2,5.0
2,8,Grand Theft Auto IV,4.0
3,10,SoulCalibur,4.0
4,11,Grand Theft Auto IV,4.5
...,...,...,...
4995,4529,Donut County,2.5
4996,4533,MotorStorm: Apocalypse,3.0
4997,4544,The Last Guy,3.0
4998,4548,Valiant Hearts: The Great War,4.0


In [5]:
#step 1: create a user-item matrix (rows: users, columns: game, values: 
user_item_matrix=data.pivot_table(index='userId', columns= 'game', values='rating')
'''
pivot_table: this function reshapes the dataframe into a matrix where:
    each row represents a user
    each column represents a game
    the values in the matrix represent the ratings that users gave to the games
'''                                  

'\npivot_table: this function reshapes the dataframe into a matrix where:\n    each row represents a user\n    each column represents a game\n    the values in the matrix represent the ratings that users gave to the games\n'

In [6]:
#step 2: fill NaN values with 0 (assuming no rating means the game has not been rated yet)
user_item_matrix_filled=user_item_matrix.fillna(0)


In [11]:
#step3: compute the cosine similarity between users based in raw ratings
user_similarity=cosine_similarity(user_item_matrix_filled)

In [15]:
#convert similarity matrix to dataframe for easier reference
user_similarity_df=pd.DataFrame(user_similarity,index=user_item_matrix.index, columns=user_item_matrix.index)

In [25]:
#step 4: fuction to get game recommendations for a specific user
def get_collaborative_recommendations_for_user(user_id,num_recommendations=5):
    #get similarity scores for the input user with all other users
    similar_users=user_similarity_df[user_id].sort_values(ascending=False)
    #get the most similar users (excluding the user themselves)
    similar_users=similar_users.drop(user_id)
    #select the top N simular users to limit noise (eg top 50 users)
    top_similar_users=similar_users.head(50)
    #get ratings of these similar users weighted by their similarity scores
    weighted_ratings= np.dot(top_similar_users.values,user_item_matrix_filled.loc[top_similar_users.index])
    #np.dot: this computed the dot product between the similarity scores of the top similar users
    #and their corresponding ratings in the user_item matrix
    #the result is an array of weighted ratings for each game

    #normalize by the sum of similarities
    sum_of_similarities=top_similar_users.sum()

    if sum_of_similarities>0:
        weighted_ratings/=sum_of_similarities
        #the weighted ratings are normalized by dividing by the sum of similarities to avoid biasing toward users with higher ratings.

    user_ratings=user_item_matrix_filled.loc[user_id]
    unrated_games=user_ratings[user_ratings==0]

    #get the weighted scores for the unrated games 
    game_recommendations=pd.Series(weighted_ratings, index=user_item_matrix_filled.columns).loc[unrated_games.index]
    #this creates a pandas series from the weighted ratings and filters to include only the unrated games
    #finally it sorts the recommended games in descending order and returns the top specified number of recommmendations

    #return the top 'num_recommendations' game recommendations
    return game_recommendations.sort_values(ascending=False).head(num_recommendations)

In [29]:
recommended_games=get_collaborative_recommendations_for_user(3)
print("Recommended games for user 3:")
print(recommended_games)

Recommended games for user 3:
game
'Splosion Man         0.0
Resogun               0.0
Resogun: Heroes       0.0
Retro City Rampage    0.0
Retro/Grade           0.0
dtype: float64
